### **Foundation**

Use Python to fetch XAUUSDc H1 candles data from MT5. UTC-based timezone. Use plotly whitethemed default candlestick chart. Identify the 'trading_daily_session' as from market-open to market-close, which includes pre-market and overnight time. Plot virtical-dashed line for every first open-candle of the 'trading_daily_session'  and the closing-candle of the 'trading_daily_session'. To easily identify the 'trading_daily_session', simply locate the very first candle recorded in pre-market and the last candle recorded at overnight.

```python

# Parameters
symbol = "XAUUSDc"
timeframe = mt5.TIMEFRAME_H1
utc_timezone = pytz.UTC
lookback_days = 10  # fetch last 10 days of H1 candles


Use Python to fetch XAUUSDc H1 candles data from MT5. UTC based timezone. Identify the 'trading_day' as from market-open to market-close, which includes pre-market and overnight session.

Identify the 'trading_day' as from market-open to market-close, which includes pre-market and overnight session.

Identify the 'trading_day' as from market-open to market-close, which includes pre-market and overnight sessions. Make a developing RSI indicator that starts calculating from the start of the trading day and resets at the close of the trading day. PLot the tivk-volume bars on separate pane, plot the RSI indicaor in seeparate pane.

import MetaTrader5 as mt5
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta, timezone

# ----------------------------
# MT5 INITIALIZATION
# ----------------------------
if not mt5.initialize():
    raise RuntimeError("MT5 initialization failed")

SYMBOL = "XAUUSDc"
TIMEFRAME = mt5.TIMEFRAME_H1
UTC = timezone.utc

# ----------------------------
# FETCH 10 DAILY CANDLES (UTC)
# ----------------------------
now_utc = datetime.now(UTC)
daily_rates = mt5.copy_rates_from(
    SYMBOL,
    mt5.TIMEFRAME_D1,
    now_utc,
    10
)

if daily_rates is None or len(daily_rates) == 0:
    mt5.shutdown()
    raise RuntimeError("No daily data retrieved")

daily_df = pd.DataFrame(daily_rates)
daily_df["time"] = pd.to_datetime(daily_df["time"], unit="s", utc=True)

start_time = daily_df["time"].min()
end_time = daily_df["time"].max() + timedelta(days=1)

# ----------------------------
# FETCH H1 DATA COVERING ALL DAYS
# ----------------------------
h1_rates = mt5.copy_rates_range(
    SYMBOL,
    TIMEFRAME,
    start_time,
    end_time
)

mt5.shutdown()

if h1_rates is None or len(h1_rates) == 0:
    raise RuntimeError("No H1 data retrieved")

df = pd.DataFrame(h1_rates)
df["time"] = pd.to_datetime(df["time"], unit="s", utc=True)
df.set_index("time", inplace=True)

# ----------------------------
# IDENTIFY TRADING DAYS
# ----------------------------
df["trading_day"] = df.index.date

session_bounds = (
    df.groupby("trading_day")
      .agg(
          first_time=("open", lambda _: df.index[df["trading_day"] == _.name][0]),
          last_time=("open", lambda _: df.index[df["trading_day"] == _.name][-1])
      )
)

# ----------------------------
# PLOTLY FIGURE (2 PANES)
# ----------------------------
fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    row_heights=[0.75, 0.25],
    subplot_titles=("XAUUSDc H1 Candles (UTC)", "Tick Volume")
)

# ----------------------------
# CANDLESTICKS
# ----------------------------
fig.add_trace(
    go.Candlestick(
        x=df.index,
        open=df["open"],
        high=df["high"],
        low=df["low"],
        close=df["close"],
        name="Price"
    ),
    row=1,
    col=1
)

# ----------------------------
# TICK VOLUME
# ----------------------------
fig.add_trace(
    go.Bar(
        x=df.index,
        y=df["tick_volume"],
        name="Tick Volume"
    ),
    row=2,
    col=1
)

# ----------------------------
# SESSION START / END LINES
# ----------------------------
for _, row in session_bounds.iterrows():
    fig.add_vline(
        x=row["first_time"],
        line=dict(color="black", width=1, dash="dash")
    )
    fig.add_vline(
        x=row["last_time"],
        line=dict(color="black", width=1, dash="dash")
    )

# ----------------------------
# LAYOUT
# ----------------------------
fig.update_layout(
    template="plotly_white",
    height=900,
    xaxis_rangeslider_visible=False,
    title="XAUUSDc – Trading Day Boundaries (UTC, Pre-Market + Overnight Included)",
    showlegend=False
)

fig.show()
